In [1]:
import pandas as pd
import json
from environs import Env
from graph_functions import query_to_df,start_graph_session, start_graph_session_crashdown
from IPython.display import Image


env = Env()
#session = start_graph_session(env) 
session = start_graph_session_crashdown(env) 

14196


In [ ]:
### List of all breast-related GWAS

In [2]:
query="""
MATCH (gwas:Gwas)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") 
and not(toLower(gwas.trait) contains "mother")
and not(toLower(gwas.trait) contains "sibling")
and not(toLower(gwas.trait) contains "mammogram")
return distinct gwas.trait, gwas.id, gwas.sample_size as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by SampleSize desc
"""
df = query_to_df(session, query)
#df.to_csv("query_results/bc_all_related_gwas.tsv", sep="\t", index=False)
df


MATCH (gwas:Gwas)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") 
and not(toLower(gwas.trait) contains "mother")
and not(toLower(gwas.trait) contains "sibling")
and not(toLower(gwas.trait) contains "mammogram")
return distinct gwas.trait, gwas.id, gwas.sample_size as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by SampleSize desc



,gwas.trait,gwas.id,SampleSize,N_case,N_control,gwas.author,gwas.year,gwas.consortium
0,Carcinoma in situ of breast,finn-a-CD2_INSITU_BREAST,None,377.0,96122.0,None,2020.0,None
1,Benign neoplasm of breast (ICD C excluded),finn-a-CD2_BENIGN_BREAST_EXALLC,None,941.0,86350.0,None,2020.0,None
2,"Carcinoma in situ of breast, intraductal",finn-a-CD2_INSITU_BREAST_INTRADUCTAL,None,334.0,53928.0,None,2020.0,None
3,Carcinoma in situ of breast (ICD C excluded),finn-a-CD2_INSITU_BREAST_EXALLC,None,377.0,87024.0,None,2020.0,None
4,Benign neoplasm of breast,finn-a-CD2_BENIGN_BREAST,None,941.0,95558.0,None,2020.0,None
...,...,...,...,...,...,...,...,...
60,ER- Breast cancer (GWAS),ieu-a-1166,21695.0,3611.0,18084.0,Michailidou K,2015.0,None
61,ER+ Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1127,175475.0,69501.0,105974.0,Michailidou K,2017.0,None
62,Breast cancer,ebi-a-GCST004988,139274.0,76192.0,63082.0,Michailidou K,2017.0,None
63,ER- Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1128,127442.0,21468.0,105974.0,Michailidou K,2017.0,None


## Get BC GWAS that have MR results

In [3]:
query="""
MATCH (gwas:Gwas)<-[mr:MR_EVE_MR]-(othergwas:Gwas)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") 
and not(toLower(gwas.trait) contains "mother")
and not(toLower(gwas.trait) contains "sibling")
and not(toLower(gwas.trait) contains "mammogram")
and not(toLower(gwas.trait) contains "protein")
return distinct gwas.trait, gwas.id, toInteger(gwas.sample_size) as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by gwas.id,  gwas.author, N_case desc
"""
df = query_to_df(session, query)
#df.to_csv("query_results/bc_gwas_with_mr.tsv", sep="\t", index=False)

# calulate cases % in the dataset and only keep those >2%
df['case_percent'] = df['N_case']/df['SampleSize']*100
df = df[df['case_percent'] > 2]


df.to_csv("query_results/bc_gwas_with_mr_keep.tsv", sep="\t", index=False)
all_bc_gwas = list(df['gwas.id'])
df



MATCH (gwas:Gwas)<-[mr:MR_EVE_MR]-(othergwas:Gwas)
where toLower(gwas.trait) contains "breast"
and not(toLower(gwas.trait) contains "baby") 
and not(toLower(gwas.trait) contains "mother")
and not(toLower(gwas.trait) contains "sibling")
and not(toLower(gwas.trait) contains "mammogram")
and not(toLower(gwas.trait) contains "protein")
return distinct gwas.trait, gwas.id, toInteger(gwas.sample_size) as SampleSize,
toInteger(gwas.ncase) as N_case, toInteger(gwas.ncontrol) as N_control, gwas.author, gwas.year, gwas.consortium
order by gwas.id,  gwas.author, N_case desc



,gwas.trait,gwas.id,SampleSize,N_case,N_control,gwas.author,gwas.year,gwas.consortium,case_percent
0,Breast cancer,ebi-a-GCST004988,139274,76192,63082,Michailidou K,2017.0,None,54.706550
1,Breast cancer,ebi-a-GCST007236,89677,46785,42892,Michailidou K,2015.0,None,52.170568
2,Breast cancer (Combined Oncoarray; iCOGS; GWAS...,ieu-a-1126,228951,122977,105974,Michailidou K,2017.0,None,53.713240
3,ER+ Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1127,175475,69501,105974,Michailidou K,2017.0,None,39.607351
4,ER- Breast cancer (Combined Oncoarray; iCOGS; ...,ieu-a-1128,127442,21468,105974,Michailidou K,2017.0,None,16.845310
5,Breast cancer (Oncoarray),ieu-a-1129,106776,61282,45494,Michailidou K,2017.0,None,57.393047
6,Breast cancer (iCOGS),ieu-a-1130,89677,46785,42892,Michailidou K,2017.0,None,52.170568
7,Breast cancer (GWAS),ieu-a-1131,32498,14910,17588,Michailidou K,2017.0,None,45.879746
8,ER+ Breast cancer (Oncoarray),ieu-a-1132,83691,38197,45494,Michailidou K,2017.0,None,45.640511
9,ER+ Breast cancer (iCOGS),ieu-a-1133,69970,27078,42892,Michailidou K,2017.0,None,38.699443


In [53]:
all_bc_gwas

['ebi-a-GCST004988',
 'ebi-a-GCST007236',
 'ieu-a-1126',
 'ieu-a-1127',
 'ieu-a-1128',
 'ieu-a-1129',
 'ieu-a-1130',
 'ieu-a-1131',
 'ieu-a-1132',
 'ieu-a-1133',
 'ieu-a-1134',
 'ieu-a-1135',
 'ieu-a-1136',
 'ieu-a-1137',
 'ieu-a-1160',
 'ieu-a-1161',
 'ieu-a-1162',
 'ieu-a-1163',
 'ieu-a-1164',
 'ieu-a-1165',
 'ieu-a-1166',
 'ieu-a-1167',
 'ieu-a-1168',
 'ukb-a-55',
 'ukb-b-16890',
 'ukb-d-C3_BREAST_3']

## MR data

In [3]:
# MR test
query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id = "ieu-a-1126"  
AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-08
RETURN exposure {.id, .trait}, outcome.id, mr {.b, .pval, .method, .selection, .moescore} ORDER BY mr.pval
"""
df = query_to_df(session, query)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id = "ieu-a-1126"  
AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-08
RETURN exposure {.id, .trait}, outcome.id, mr {.b, .pval, .method, .selection, .moescore} ORDER BY mr.pval



,outcome.id,exposure.trait,exposure.id,mr.b,mr.method,mr.selection,mr.pval,mr.moescore
0,ieu-a-1126,Ever addicted to alcohol,ukb-d-20406,-0.054825,FE IVW,DF,0.000000e+00,1.00
1,ieu-a-1126,Diagnoses - secondary ICD10: K44.9 Diaphragmat...,ukb-b-3672,-1.542995,FE IVW,DF,4.514856e-242,1.00
2,ieu-a-1126,Killer cell immunoglobulin-like receptor 2DL5A,prot-a-1644,0.016679,FE IVW,DF,4.770365e-106,1.00
3,ieu-a-1126,Ever had hysterectomy (womb removed),ukb-b-6445,-1.562231,FE IVW,DF,9.686474e-86,1.00
4,ieu-a-1126,Operation code: bunion/hallus valgus surgery,ukb-b-9190,1.109370,FE IVW,DF,3.334299e-85,1.00
5,ieu-a-1126,"Vascular endothelial growth factor A, isoform 121",prot-a-3197,0.013410,FE IVW,DF,7.992747e-83,1.00
6,ieu-a-1126,fatty acid binding protein 4,prot-b-71,0.019159,FE IVW,DF,1.571455e-58,1.00
7,ieu-a-1126,Diagnoses - secondary ICD10: Z92.1 Personal hi...,ukb-b-7352,-0.973106,FE IVW,DF,1.267170e-57,1.00
8,ieu-a-1126,CUB and zona pellucida-like domain-containing ...,prot-a-735,0.021479,FE IVW,DF,2.941390e-37,1.00
9,ieu-a-1126,Filamin-A,prot-a-1117,-0.054938,FE IVW,DF,9.719336e-26,1.00


In [6]:
# MR
# get traits that have pval < 1e-05

query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list} 
AND  not exposure.id  in {gwas_list}
//AND (not (toLower(exposure.trait) contains 'breast')) 
AND mr.pval < 1e-05
with mr, exposure, outcome
ORDER BY mr.pval 
RETURN exposure.id, exposure.trait, exposure.sample_size,
        collect(outcome.id) as outcome_ids, 
        collect(mr.pval) as MR_pvals, collect(mr.b) as MR_beta
        
""".format(gwas_list = all_bc_gwas)
df = query_to_df(session, query, False)

df

,exposure.id,exposure.trait,exposure.sample_size,outcome_ids,MR_pvals,MR_beta
0,ukb-b-323,Illnesses of father: Alzheimer's disease/dementia,399793.0,"[ebi-a-GCST004988, ieu-a-1134, ieu-a-1132, ieu...","[0.0, 8.299326114712149e-72, 6.21025571093903e...","[-0.930140957122164, 1.69433355372966, -1.5471..."
1,prot-a-408,C-C motif chemokine 3-like 1,3301.0,"[ebi-a-GCST004988, ebi-a-GCST007236, ieu-a-113...","[0.0, 5.37437935276178e-45, 3.2070884984982397...","[0.0172084678909587, 1.39602294061095, 0.07991..."
2,ukb-d-H40,Diagnoses - main ICD10: H40 Glaucoma,361194.0,"[ebi-a-GCST007236, ieu-a-1137]","[0.0, 4.4397636534247396e-11]","[-726.001285117886, 24.913334445577696]"
3,ukb-d-30270_irnt,Mean sphered cell volume,344729.0,[ebi-a-GCST007236],[0.0],[7.490779645720821]
4,ukb-b-2732,Time from waking to first cigarette,33304.0,"[ebi-a-GCST007236, ukb-a-55]","[0.0, 1.0627484509411099e-09]","[-11.1357984372859, 0.00667248437247834]"
...,...,...,...,...,...,...
1449,met-a-359,Gamma-glutamylglutamine,7662.0,[ieu-a-1167],[7.44948187880931e-06],[-1.7471772915958101]
1450,ukb-b-19809,Fed-up feelings,453071.0,[ieu-a-1161],[7.94247770454966e-06],[1.2684340568579098]
1451,prot-a-821,Dickkopf-related protein 1,3301.0,[ieu-a-1136],[8.24099097862065e-06],[0.0339189126254016]
1452,ukb-a-361,Heel bone mineral density (BMD) T-score autom...,106254.0,[ebi-a-GCST007236],[8.33635583806819e-06],[-5.32572418764976]


In [9]:
# now those traits that appered at least once in something at <1e05, 
# get MR results for those traits with all BC datasets


query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list} 
AND  exposure.id  in {exposure_list}
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, exposure.sample_size, exposure.sex, exposure.note,
toInteger(exposure.year) as year, exposure.author as author, exposure.consortium as consortium,
        outcome.id, outcome.sample_size, toInteger(outcome.ncase) as N_case, outcome.year,
        mr.pval, mr.b, mr.se, mr.method, mr.moescore
""".format(gwas_list = all_bc_gwas,
           exposure_list = list(set(df['exposure.id'])))
df = query_to_df(session, query, True)


df.to_csv("query_results/bc_all_mr.tsv", sep="\t", index=False)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ebi-a-GCST004988', 'ebi-a-GCST007236', 'ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1131', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1134', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1137', 'ieu-a-1160', 'ieu-a-1161', 'ieu-a-1162', 'ieu-a-1163', 'ieu-a-1164', 'ieu-a-1165', 'ieu-a-1166', 'ieu-a-1167', 'ieu-a-1168', 'ukb-a-55', 'ukb-b-16890', 'ukb-d-C3_BREAST_3'] 
AND  exposure.id  in ['ukb-b-11158', 'prot-a-2427', 'ukb-d-I9_DVTANDPULM', 'prot-a-2351', 'ieu-a-65', 'ukb-b-16858', 'ebi-a-GCST004941', 'prot-a-1276', 'ebi-a-GCST006475', 'prot-a-2061', 'ieu-a-27', 'ebi-a-GCST005838', 'ieu-a-1228', 'ukb-b-845', 'ukb-b-4575', 'ukb-d-30050_irnt', 'ebi-a-GCST006095', 'ubm-a-791', 'prot-a-1202', 'ukb-b-4171', 'ebi-a-GCST003045', 'prot-a-2243', 'ukb-d-R79', 'ieu-a-73', 'ebi-a-GCST005527', 'ukb-a-235', 'prot-a-216', 'prot-a-2907', 'ubm-a-29', 'prot-a-774', 'ukb-b-11358', 'ukb-b-5447', 'ukb-b-9487', 'ukb-a-45

,exposure.id,exposure.trait,exposure.sample_size,exposure.sex,exposure.note,year,author,consortium,outcome.id,outcome.sample_size,N_case,outcome.year,mr.pval,mr.b,mr.se,mr.method,mr.moescore
0,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1165,37954.0,2900,2015.0,0.500000,0.124119,0.124119,Wald ratio,1.0
1,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1127,175475.0,69501,2017.0,0.012762,-4.294414,1.724423,FE IVW,1.0
2,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1128,127442.0,21468,2017.0,0.780908,0.885429,3.183436,FE IVW,1.0
3,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ukb-a-55,337159.0,7480,2017.0,0.569689,-0.030093,0.052933,FE IVW,1.0
4,ukb-d-M13_POLYARTHROPATHIES,#Polyarthropathies,361194.0,Males and Females,None,2018,Neale lab,None,ieu-a-1126,228951.0,122977,2017.0,0.192254,-2.230371,1.710487,FE IVW,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35109,ubm-a-2685,volume Right-VentralDC,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1131,32498.0,14910,2017.0,0.821754,-0.039192,0.173963,FE IVW,1.0
35110,ubm-a-2685,volume Right-VentralDC,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1136,50225.0,7333,2017.0,0.463249,0.153854,0.209752,FE IVW,1.0
35111,ubm-a-2685,volume Right-VentralDC,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1129,106776.0,61282,2017.0,0.230856,0.069494,0.058001,FE IVW,1.0
35112,ubm-a-2685,volume Right-VentralDC,8411.0,Males,None,2018,Elliott LT,None,ieu-a-1130,89677.0,46785,2017.0,0.733248,0.018464,0.054178,FE IVW,1.0


### Extract MR results for a list of exposure GWAS


In [6]:
# read list of exp gwas (EXTRACTED IN R)
risk_factors = list(pd.read_csv("query_results/gwas_rf_list.txt", sep = '\t')['id'])


query = """
MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in {gwas_list_outcome} 
AND  exposure.id  in {gwas_list_exposure}
//AND mr.pval < 1e-05
with mr, exposure, outcome
ORDER BY exposure.trait
RETURN exposure.id, exposure.trait, exposure.sample_size, exposure.sex,exposure.note,
        outcome.id, outcome.sample_size, outcome.year,
        toInteger(outcome.ncase) as N_case,
        mr.pval, mr.b, mr.se, mr.method, mr.moescore
        
""".format(gwas_list_outcome = all_bc_gwas,
           gwas_list_exposure = risk_factors )
df = query_to_df(session, query, True)


df.to_csv("query_results/gwas_rf_bc_all_mr.tsv", sep="\t", index=False)
df


MATCH (exposure:Gwas)-[mr:MR_EVE_MR]->(outcome:Gwas)
WHERE outcome.id in ['ukb-a-519', 'ukb-a-55', 'ieu-a-1126', 'ieu-a-1127', 'ieu-a-1128', 'ieu-a-1129', 'ieu-a-1130', 'ieu-a-1132', 'ieu-a-1133', 'ieu-a-1135', 'ieu-a-1136', 'ieu-a-1131', 'ieu-a-1137', 'ieu-a-1134'] 
AND  exposure.id  in ['ukb-b-8764', 'ukb-b-7663', 'ukb-a-485', 'ukb-b-2115', 'ukb-b-13932', 'ukb-b-13184', 'ukb-a-483', 'ukb-a-482', 'ukb-a-487', 'ukb-a-512', 'ukb-b-15869', 'ukb-b-11495', 'ukb-b-8865', 'ukb-a-509', 'ukb-a-486', 'ukb-b-2346', 'ukb-b-7337', 'ukb-a-484', 'ukb-b-13702', 'ieu-a-1095', 'ieu-a-1004', 'ukb-b-4206', 'ukb-a-320', 'ukb-b-18541', 'ukb-a-317', 'ukb-b-1209', 'ukb-b-18105', 'ukb-b-17422', 'ukb-a-315', 'ukb-b-9547', 'ukb-b-12405', 'ukb-a-451', 'ukb-b-8727', 'ukb-b-4564', 'ukb-b-8080', 'ukb-a-322', 'ukb-b-9433', 'ukb-a-324', 'ukb-b-4193', 'ukb-b-3768', 'ukb-a-319', 'ukb-a-316', 'ukb-a-323', 'ukb-b-9509', 'ukb-b-16927', 'ukb-b-9487', 'ieu-a-29', 'ieu-a-27', 'ukb-b-13378', 'ieu-b-40', 'ieu-a-974', 'ukb-b-1

,exposure.id,exposure.trait,exposure.sample_size,exposure.sex,outcome.id,outcome.sample_size,N_case,mr.pval,mr.b,mr.se
0,ukb-a-319,Age at first live birth,123846.0,Males and Females,ukb-a-519,337199.0,5510,0.558758,0.001719,0.002879
1,ukb-a-319,Age at first live birth,123846.0,Males and Females,ukb-a-55,337159.0,7480,0.816717,0.001187,0.005120
2,ukb-a-319,Age at first live birth,123846.0,Males and Females,ieu-a-1126,228951.0,122977,0.013006,-0.262657,0.105756
3,ukb-a-319,Age at first live birth,123846.0,Males and Females,ieu-a-1127,175475.0,69501,0.789003,-0.034011,0.124982
4,ukb-a-319,Age at first live birth,123846.0,Males and Females,ieu-a-1128,127442.0,21468,0.618318,-0.171930,0.337429
...,...,...,...,...,...,...,...,...,...,...
596,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1136,50225.0,7333,0.000697,-0.342825,0.100251
597,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1131,32498.0,14910,0.042964,-0.177672,0.087480
598,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1130,89677.0,46785,0.582227,0.038816,0.070557
599,ukb-a-266,Whole body fat-free mass,331291.0,Males and Females,ieu-a-1132,83691.0,38197,0.862899,0.007952,0.046012


### Correlation 

In [7]:
# find which traits are highly correlated with UKB cancers

query = """
MATCH (trait:Gwas)-[gen_cor:GEN_COR]-(assoc_trait:Gwas)
where assoc_trait.id in ['ukb-a-519']
match (trait:Gwas)-[mr:MR_EVE_MR]->(assoc_trait:Gwas)
//where mr.pval < 1e-03
return distinct trait.trait, trait.id, gen_cor.rg, assoc_trait.trait, assoc_trait.id, mr.b, mr.pval
order by gen_cor.rg desc
"""
df = query_to_df(session, query)
df





MATCH (trait:Gwas)-[gen_cor:GEN_COR]-(assoc_trait:Gwas)
where assoc_trait.id in ['ukb-a-519']
match (trait:Gwas)-[mr:MR_EVE_MR]->(assoc_trait:Gwas)
//where mr.pval < 1e-03
return distinct trait.trait, trait.id, gen_cor.rg, assoc_trait.trait, assoc_trait.id, mr.b, mr.pval
order by gen_cor.rg desc



,trait.trait,trait.id,gen_cor.rg,assoc_trait.trait,assoc_trait.id,mr.b,mr.pval
0,Number of self-reported cancers,ukb-a-21,0.8279,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.077594,1.850559e-02
1,Had other major operations,ukb-a-326,0.2660,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.026946,5.000000e-01
2,Breastfed as a baby,ukb-a-33,0.2471,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.012162,5.060810e-01
3,Headaches for 3+ months,ukb-a-354,0.1980,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.017729,3.182665e-29
4,Qualifications: NVQ or HND or HNC or equivalent,ukb-a-402,0.1885,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,0.026366,5.000000e-01
...,...,...,...,...,...,...,...
260,Reason for glasses/contact lenses: For 'astigm...,ukb-a-422,-0.1279,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.077385,5.000000e-01
261,Diagnoses - main ICD10: I21 Acute myocardial i...,ukb-a-533,-0.1304,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.057651,5.647018e-01
262,Number of days/week of vigorous physical activ...,ukb-a-511,-0.1306,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.006864,3.821609e-02
263,Mineral and other dietary supplements: None of...,ukb-a-493,-0.1405,Diagnoses - main ICD10: C50 Malignant neoplasm...,ukb-a-519,-0.014608,6.539805e-01
